## Region of interest

Path to geojson files of cities, Toulouse, Cusco, and Calama

In [ ]:
# path_to_roi_geojson = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\roi\chengdu-china.geojson'
# path_to_roi_geojson = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\roi\calama.geojson'
# path_to_roi_geojson = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\roi\cusco.geojson'
path_to_roi_geojson = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\roi\toulouse.geojson'

Load ROI from geojson file

In [ ]:
import json
f = open(path_to_roi_geojson)
dat = json.load(f)
#roi = dat['geometry']['coordinates']
roi = dat['coordinates']
dat

In [ ]:
loc_name = dat['properties']['title']
import numpy as np
import ee
ee.Initialize()
def get_utm_epsg_ee(lon, lat):
    #coords = feature.geometry().coordinates()
    lon = ee.Number(lon)
    lat = ee.Number(lat)
    epsg = ee.Number(32700).subtract(lat.add(45).divide(90).round().multiply(100)).add(lon.add(183).divide(6).round()).uint16()    
    return ee.String("EPSG:").cat(ee.String(str(epsg.getInfo())))#ee.String("EPSG:").cat
def get_utm_epsg(lon,lat):
    if lat<0:
        EPSG = 32700
    else:
        EPSG = 32600
    EPSG = int(EPSG + np.floor((lon+180)/6) + 1)
    return EPSG

ee_roi = ee.Geometry.Polygon(roi)
epsg=get_utm_epsg_ee(roi[0][0][0], roi[0][0][1])
print(epsg.getInfo())



Initial directory for saving data

In [ ]:
import os

def initial_dir_if_not_exits(directory):
    if not os.path.exists(directory):
        upper_dir = '\\'.join(directory.split('\\')[:-1])
        print(upper_dir)
        if os.path.exists(upper_dir):                 
            os.mkdir(directory)
        else:
            initial_dir_if_not_exits(upper_dir)
    return 0



path_to_data_store_directory = r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\Sentinel-2'
loc_dir = os.path.join(path_to_data_store_directory,loc_name)
initial_dir_if_not_exits(loc_dir)


Visualize the ROI on a base map

In [ ]:
import geemap
Map = geemap.Map()
Map.centerObject(ee_roi,13)
Map.addLayer(ee_roi, {'color': 'FF0000'})
Map

## Download all S2 images monthly

import ee
import os
def clipImg(x):
    return x.clip(ee_roi)
s2_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
years = [2018,2019]
months = [1,2,3,4,5,6,7,8,9,10,11,12]


for y in years:
    for m in months:
        # initial a directory for data of every month
        if m<10:
            time_str = str(y)+'_0'+str(m)
        else:
            time_str = str(y)+'_'+str(m)        
        monthly_dir = os.path.join(loc_dir, time_str)
        initial_dir_if_not_exits(monthly_dir)
        
        # set time period and retrieve corresponding data
        start_date = ee.Date.fromYMD(y, m, 1)
        if m == 12:
            end_date = ee.Date.fromYMD(y+1, 1, 1)
        else:
            end_date = ee.Date.fromYMD(y, m+1, 1)
            
        collection = ee.ImageCollection('COPERNICUS/S2') \
            .filterBounds(ee_roi) \
            .filterDate(start_date, end_date) \
            .map(clipImg) \
            .select(s2_bands)
        
        # download data sets
        geemap.ee_export_image_collection(collection, out_dir=monthly_dir, region=ee_roi, scale=10)



## Generate one cloud free data per month, and download

In [ ]:
import sys
sys.path.append(r'C:\Users\Jingliang\Documents\projects\Sentinel-1_time_series\Extra_cities\src')
import sentinel_2_cloud_free

import ee
import os
def clipImg(x):
    return x.clip(ee_roi)
years = [2018,2019]
months = [1,2,3,4,5,6,7,8,9,10,11,12]

# years = [2018]
# months = [1]

for y in years:
    for m in months:
        # initial a directory for data of every month
        if m<10:
            time_str = str(y)+'_0'+str(m)
        else:
            time_str = str(y)+'_'+str(m)
            
        monthly_dir = os.path.join(loc_dir, time_str)
        initial_dir_if_not_exits(monthly_dir)
        
        # set time period and retrieve corresponding data
        start_date = ee.Date.fromYMD(y, m, 1)
        if m == 12:
            end_date = ee.Date.fromYMD(y+1, 1, 1)
        else:
            end_date = ee.Date.fromYMD(y, m+1, 1)
            
            
        # This function solve everything, but the data can not be downloaded
        img = sentinel_2_cloud_free.exportCloudFreeSen2(ee_roi, start_date, end_date)
        
        # Visualize the cloudy free data in map
        Map.addLayer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000})
        
        # download data sets
        # geemap.ee_export_image_collection(collection, out_dir=monthly_dir, region=ee_roi, scale=10, file_per_band=True)
        geemap.ee_export_image(img, filename=os.path.join(monthly_dir,'cloud_free.tif'), scale=10, region=ee_roi, crs=ee.Projection(epsg), file_per_band=True)
